In [11]:
import numpy as np
import cv2 as cv
import glob



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (24,17)
frameSize = (600,400)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
print(objp)
size_of_chessboard_squares_mm = 20
objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('images/chessboards/*.png')

for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        # corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Print the number of corners found
        print(f"Corners found in image: {len(corners)}")

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners, ret)
        cv.imshow('img', img)
        cv.waitKey(300)
    else:
        print('Not found')


cv.destroyAllWindows()




[[ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 2.  0.  0.]
 ...
 [21. 16.  0.]
 [22. 16.  0.]
 [23. 16.  0.]]
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408
Corners found in image: 408


#### What is objp?
*objp* is an array containing the 3D world coordinates of the corners in the chessboard used for calibration. It's used to map the chessboard corners in the real world to the image plane detected by cv.findChessboardCorners.

Computation of *objp*:
Initialization: The array is initialized to have a size where the first dimension is equal to the total number of corners (product of chessboard dimensions minus one on each side, but in this code, you seem to be using the full grid dimensions chessboardSize, which suggests the use of an intersection points grid rather than an inner corners grid).

```python
    objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
```
This creates a flat list to store coordinates for each corner, initialized to zeros.

Grid Coordinates Setup: The numpy.mgrid function generates a mesh grid which is then used to set the x and y coordinates. This is reshaped to fit the first two columns of objp, essentially assigning each corner a coordinate in this grid:

```python
objp[:, :2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2)
```
Here, np.mgrid creates two 2D arrays for x (rows) and y (columns) coordinates that correspond to the indices of the grid (i.e., 0, 1, 2, ..., N where N is one less than the number of squares along each dimension of the chessboard). .T transposes the arrays so that coordinates line up correctly in the reshape call, which flattens them into a two-column matrix for all points.

Scale by Physical Size: Finally, each coordinate is scaled by size_of_chessboard_squares_mm, which represents the physical size of each square on the chessboard:

```python
objp = objp * size_of_chessboard_squares_mm
```
This multiplication scales the grid so that the distances between points reflect the actual size of the chessboard squares in millimeters. The z-coordinate remains zero, assuming the chessboard is flat on a surface.

In [12]:

############## CALIBRATION #######################################################
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)
print(dist)

[[-2.43197723e-01  1.32016607e-01  7.87084956e-04  1.35990754e-04
  -5.18071839e-02]]


In [13]:
############## UNDISTORTION #####################################################

img = cv.imread('images/chessboards/Image__2018-10-05__10-30-50.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imshow('img', dst)
cv.waitKey(400)
cv.destroyAllWindows()


In [14]:
# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv.imshow('img', dst)
cv.waitKey(400)
cv.destroyAllWindows()



In [16]:
# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print(len(objpoints))
print( "total error: {}".format(mean_error/len(objpoints)) )

13
total error: 0.018073245696731074
